In [39]:
import numpy as np

In [5]:
# Element class
class Element(object):
    
    def __init__(self,position,dimensions,rotation=0.0,density=0.0):
        '''
        Set up
        '''
        self.postion = position
        self.dimensions = dimensions
        self.rotation = rotation
        self.density = density
        self.loadNodes = []
        self.supportNodes = []
        
    def addLoadNode(self,loadNode):
        self.loadNodes.append(loadNode)
        # if this treats a non-const reference, it should set up the node relationship too
        
    def addSupportNode(self,supportNode):
        self.supportNodes.append(supportNode)
        # if this treats a non-const reference, it should set up the node relationship too
        
    def setSelfLoadOnNodes(self):
        raise NotImplementedError()
        

In [50]:
# Node class
class Node(object):
    
    def __init__(self, position):
        '''
        Set up
        '''
        self.position = position
        self.fixMoment = [True, True, True]
        self.fixForce = [True, True, True]
        self.externalMoment = [0.0, 0.0, 0.0]
        self.externalForce = [0.0, 0.0, 0.0]
        self.actingMoment = [0.0, 0.0, 0.0]
        self.actingForce = [0.0, 0.0, 0.0]
        self.loadElements = []
        self.supportElements = []
        
    def addLoadElement(self,loadElement):
        self.loadElements.append(loadElement)
        # if this treats a non-const reference, it should set up the element relationship too
        
    def addSupportElement(self,supportElement):
        self.supportElements.append(supportElement)
        # if this treats a non-const reference, it should set up the element relationship too
    
    def calcReactForce(self):
        #todo(ajm 10/2/2019): need DOF logic
        
        reactForce = [0.0, 0.0, 0.0]
        for comp, force in enumerate(self.externalForce):
            reactForce[comp] -= force
        for loadElement in self.loadElements:
            for loadNode in loadElement.loadNodes:
                for comp, force in enumerate(loadNode.actingForce):
                    reactForce[comp] -= force
                    
        return reactForce
    
    def setActingForce(self):
        reactForce = self.calcReactForce()
        self.actingForce = [-f for f in reactForce]
            
    def calcReactMoment(self):
        #todo(ajm 10/2/2019): need DOF logic
        
        reactMoment = [0.0, 0.0, 0.0]

        for jdx,moment in enumerate(self.externalMoment):
            reactMoment[jdx] -= moment
        
        for loadElement in self.loadElements:
            for loadNode in loadElement.loadNodes:
                distVect = [x-y for x,y in zip(loadNode.position,self.position)]
                momentVect = np.cross(distVect,loadNode.actingForce)
                for jdx,moment in enumerate(momentVect):
                    reactMoment[jdx] -= moment
                    reactMoment[jdx] -= loadNode.actingMoment[jdx]

        return reactMoment
    
    def setActingMoment(self):
        # todo (ajm 10/2/2019): careful, this might be over simplified...
        reactMoment = self.calcReactMoment()
        self.actingMoment = [-m for m in reactMoment]

In [48]:
'''
Test #1

    ^ 18N
    |
 c2 x --> 20N 
    |
 b1 |
    |
 c1 x
'''

# define components
conn1 = Node([0.0, 0.0, 0.0])
conn2 = Node([0.0, 3.0, 0.0])
beam1 = Element([0.0, 0.0, 0.0], {"length":3.0}, 90.0)

# define relationships
beam1.addLoadNode(conn2)
beam1.addSupportNode(conn1)

conn2.addSupportElement(beam1)
conn1.addLoadElement(beam1)

# define load
conn2.externalForce = [20.0, 18.0, 0.0]

# communicate loads
conn2.setActingForce()

# check answers
print("should be (-20, -18, 0): ",conn1.calcReactForce())
print("should be (0, 0, 60): ",conn1.calcReactMoment())

should be (-20, -18, 0):  [-20.0, -18.0, 0.0]
should be (0, 0, 60):  [0.0, 0.0, 60.0]


In [52]:
2*6-5*14-5*3+5*1.23

-66.85

In [51]:
'''
Test #1
               ^6N
               |
            c4 x --> 14N
      | 1.23N /
      v  b2  / b3
 5N-->x-----x c2
      c3    |
         b1 |
            |
         c1 x
'''

# define components
conn1 = Node([0.0, 0.0, 0.0])
conn2 = Node([0.0, 3.0, 0.0])
conn3 = Node([-5.0, 3.0, 0.0])
conn4 = Node([2.0, 5.0, 0.0])
beam1 = Element([0.0, 0.0, 0.0], {"length":3.0}, 90.0)
beam2 = Element([0.0, 3.0, 0.0], {"length":5.0}, 180.0)
beam3 = Element([0.0, 3.0, 0.0], {"length":2*2**0.5}, 45.0)

# define relationships
beam3.addLoadNode(conn4)
beam3.addSupportNode(conn2)
beam2.addLoadNode(conn3)
beam2.addSupportNode(conn2)
beam1.addLoadNode(conn2)
beam1.addSupportNode(conn1)

conn4.addSupportElement(beam3)
conn3.addSupportElement(beam2)
conn2.addLoadElement(beam3)
conn2.addLoadElement(beam2)
conn2.addSupportElement(beam1)
conn1.addLoadElement(beam1)

# define loads
conn3.externalForce = [5.0, -1.23, 0.0]
conn4.externalForce = [14.0, 6.0, 0.0]

# communicate loads
conn4.setActingForce()
conn3.setActingForce()
conn2.setActingForce()

# communicate moments
conn2.setActingMoment()

# check answers
print("should be (-19, -4.77, 0): ", conn1.calcReactForce())
print("should be (0, 0, 66.85): ", conn1.calcReactMoment())

[-19.0, -4.77, 0.0]
[0.0, 0.0, 66.85]


In [40]:
np.cross([2,6,0],[3,-2,0])

array([  0,   0, -22])